<a href="https://colab.research.google.com/github/Shiv1143/Machine-Learning-From-Scratch/blob/main/cnn_with_pytorch_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [2]:
import torch
from torchvision import datasets as dt
from torchvision import transforms as T
from torch.utils.data import DataLoader as DL
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import models
import torch.nn.functional as F

# Accessing GPU

In [3]:
# Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on device: {device}")

Training on device: cuda:0


# Load CIFAR-10 dataset

In [4]:
transform = T.Compose(
                     [T.ToTensor(),
                      T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
                     )
trainset = dt.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DL(trainset, batch_size=4, shuffle=True)

testset = dt.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DL(testset, batch_size=4, shuffle=False)


100%|██████████| 170498071/170498071 [00:02<00:00, 76812777.98it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Computing Accuracy

In [29]:
def compute_accuracy(data_loader, model, device):
    correct = 0
    total = 0
    data_iter = iter(data_loader)
    with torch.no_grad():
      while True:
          try:
              data = next(data_iter)
          except StopIteration:
              break
          inputs, labels = data[0].to(device), data[1].to(device)
          outputs = model(inputs)
          _, predicted = torch.max(outputs, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
      accuracy = (correct / total) * 100
      print(f'Accuracy on the dataset: {accuracy:.2f}%')
    return accuracy

# A simple CNN architecture

In [30]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.convolution1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
        self.convolution2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.pooling = nn.MaxPool2d(2, 2)
        self.fully_connected1 = nn.Linear(64 * 8 * 8, 120)
        self.fully_connected2 = nn.Linear(120, 60)
        self.fully_connected3 = nn.Linear(60, 10)

    def forward(self, l):
        l = self.pooling(torch.relu(self.convolution1(l)))
        l = self.pooling(torch.relu(self.convolution2(l)))
        l = l.view(-1, 64 * 8 * 8)
        l = torch.relu(self.fully_connected1(l))
        l = torch.relu(self.fully_connected2(l))
        l = self.fully_connected3(l)
        return l



# Initializing
base_net = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(base_net.parameters(), lr=0.0009, momentum=0.8)


# Training the basic cnn
for e in range(10):
    cur_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = base_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        cur_loss += loss.item()
    print(f'Epoch_count: {e + 1}, loss: {cur_loss / len(trainloader)}')

print('Training completed')

compute_accuracy(testloader, basic_net, device)

Epoch_count: 1, loss: 1.6594646263074875
Epoch_count: 2, loss: 1.1670529314103724
Epoch_count: 3, loss: 0.9399758194772899
Epoch_count: 4, loss: 0.7964017690592632
Epoch_count: 5, loss: 0.6794104351979681
Epoch_count: 6, loss: 0.5733379821524397
Epoch_count: 7, loss: 0.48060059355254287
Epoch_count: 8, loss: 0.3885987324144511
Epoch_count: 9, loss: 0.3103704693638094
Epoch_count: 10, loss: 0.24507741102138955
Training completed
Accuracy on the dataset: 25.86%


25.86

# Performance Improvement

Increase the Depth of the Network: Add more convolutional layers to increase the network's capacity to learn more complex features.

Batch Normalization: Apply batch normalization after each convolutional layer to stabilize learning and improve convergence speed.

Adjust Hyperparameters:
Used Adam optimizer instead of SGD.
Increase the learning rate initially and then use a learning rate scheduler to decrease it gradually for better convergence.
Increase the batch size if the hardware allows, to make the gradient descent smoother.


Regularization:
Add dropout layers to prevent overfitting.
Consider using L2 regularization in the optimizer for further control over model complexity.
Use a More Complex Fully Connected Layer: Increase the number of nodes in the hidden layers of the fully connected part to allow the model to learn more complex relationships.

In [31]:
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        # Feature extractor layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Classifier layers
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, l):
        # Feature extraction
        l = F.relu(self.bn1(self.conv1(l)))
        l = F.relu(self.bn2(self.conv2(l)))
        l = self.pool1(l)
        l = F.relu(self.bn3(self.conv3(l)))
        l = F.relu(self.bn4(self.conv4(l)))
        l = self.pool2(l)

        # Flatten the feature maps
        l = l.view(-1, 128 * 8 * 8)

        # Classification
        l = F.dropout(l, p=0.3, training=self.training)
        l = F.relu(self.fc1(l))
        l = F.dropout(l, p=0.3, training=self.training)
        l = self.fc2(l)
        return l

# Initializing the improved network and optimizer
improved_net = ImprovedCNN()

improved_net.to(device)

optimizer = optim.Adam(improved_net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=20, gamma=0.7)

# Model Training
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = improved_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, loss: {running_loss / len(trainloader)}')

compute_accuracy(testloader, improved_net, device)

Epoch 1, loss: 1.638599201310873
Epoch 2, loss: 1.1891960345435142
Epoch 3, loss: 0.9882375029551238
Epoch 4, loss: 0.8614704246436059
Epoch 5, loss: 0.7721889505889593
Epoch 6, loss: 0.7042082504128875
Epoch 7, loss: 0.6516130230579062
Epoch 8, loss: 0.5964381159749854
Epoch 9, loss: 0.5529211454967433
Epoch 10, loss: 0.5139980134486917
Accuracy on the dataset: 75.68%


75.68

The effectiveness of the model was significantly enhanced by adding extra layers, incorporating batch normalization, and using dropout for regularization. Additionally, changing the optimizer to Adam, increasing the batch size, and reducing the learning rate from 0.001 to 0.0001 contributed to this improvement. It should be noted that the implementation of a learning rate scheduler also improved the training process. Ultimately, these adjustments resulted in a notable improvement in the model's accuracy on test data, increasing from 25.86% to 75.68% over the same 10 epochs.

# Applying Transfer Learning

In [28]:
# Define a new classifier
class TransferLearningCNN(nn.Module):
    def __init__(self):
        super(TransferLearningCNN, self).__init__()
        # Load a pre-trained ResNet model
        self.resnet = models.resnet18(pretrained=True)
        # Freeze all the parameters in the pre-trained model
        for param in self.resnet.parameters():
            param.requires_grad = False
        # Replace the last fully connected layer with a new one suitable for CIFAR-10
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.resnet(x)

# Initialize the transfer learning model and move it to the device
transfer_learning_net = TransferLearningCNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transfer_learning_net.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Learning rate scheduler

# Training loop
for e in range(10):
    cur_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = transfer_learning_net(inputs)
        # Calculate loss
        loss = criterion(outputs, labels)
        # Backward pass
        loss.backward()
        # Optimize
        optimizer.step()

        cur_loss += loss.item()
    print(f'Epoch {e + 1}, loss: {cur_loss / len(trainloader)}')
    scheduler.step()

print('Finished Training')

# Evaluate the model
compute_accuracy(testloader, transfer_learning_net, device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 135MB/s]


Epoch 1, loss: 2.0779506033468245
Epoch 2, loss: 2.0420168658685682
Epoch 3, loss: 2.038652170205116
Epoch 4, loss: 2.0367823679971693
Epoch 5, loss: 2.038052363886833
Epoch 6, loss: 1.9675544601106643
Epoch 7, loss: 1.9618855648040772
Epoch 8, loss: 1.9470566801214217
Epoch 9, loss: 1.9423738677358626
Epoch 10, loss: 1.9393128563356399
Finished Training
Accuracy on the test images: 30.49 %
